In [ ]:
# descompactando os arquivos zip
import zipfile
with zipfile.ZipFile('/content/elon_musk_2015_2020.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile('/content/Company_Tweet.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile('/content/Tweet.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')



In [ ]:
# armazenando as tabelas em variaveis
import pandas as pd
stocks_tesla = pd.read_csv('base_bolsa.csv')
elon_tweets = pd.read_csv('tweets_elon_musk.csv')
tweets_stocks= pd.read_csv('Company_Tweet.csv')
tweets= pd.read_csv('Tweet.csv')
stocks= pd.read_csv('CompanyValues.csv')

In [ ]:
#renomear as colunas das ações da tesla e adicionar a variação
stocks_tesla.rename(columns = {'Date':'day_date', 'Close/Last':'close_value', 'Open':'open_value', 'Volume': 'volume', 'Low':'low_value', 'High': 'high_value'}, inplace = True)
stocks_tesla['variation'] = stocks['close_value'] - stocks['open_value']

In [ ]:
#renomear as colunas das ações das top 10 empresas e adicionar a variação
stocks = stocks.sort_values("day_date", ascending= True)
stocks['variation'] = stocks['close_value'] - stocks['open_value']
stocks = stocks.loc[stocks['ticker_symbol'] == 'TSLA']
stocks.drop('ticker_symbol', inplace=True, axis=1)

In [ ]:
#ordenando as ações das top 10 empresas pelo tweet_id
tweets_stocks = tweets_stocks.sort_values("tweet_id", ascending= False)
tweets = tweets.sort_values("tweet_id", ascending= False)
tweets = tweets.set_index("tweet_id")
tweets.drop('writer',inplace=True, axis=1)

In [ ]:
#retirando colunas inutilizadas nos tweets do elon musk
elon_tweets.drop('date', inplace=True,axis=1)
elon_tweets.drop('date_day', inplace=True,axis=1)
elon_tweets.drop('hour_minutes', inplace=True,axis=1)
elon_tweets.drop('hour', inplace=True,axis=1)
elon_tweets.drop('username', inplace=True,axis=1)
elon_tweets = elon_tweets.sort_values("date_new", ascending=True)

In [ ]:
#relacionando as ações com as respectivas empresas
teste = tweets_stocks['tweet_id'].tolist()
tweets_by_company = tweets.loc[teste]
tweets_by_company['idx'] = tweets_stocks.index
tweets_by_company = tweets_by_company.set_index("idx")
tweets_stocks['tweet'] = tweets_by_company['body']
tweets_stocks['date'] = tweets_by_company['post_date']
tweets_stocks['like'] = tweets_by_company['like_num']
tweets_stocks['comment'] = tweets_by_company['comment_num']
tweets_stocks['rt'] = tweets_by_company['retweet_num']
tweets_stocks['date'] = tweets_stocks['date']

In [ ]:
#ajustando um problema de compatibilidade entre as datas das ações e os tweets
import datetime
dateList = tweets_stocks['date'].tolist()
output = []
for date in dateList:
  if int(datetime.datetime.fromtimestamp(date).strftime("%H"))>=17:
    out_date = datetime.datetime.fromtimestamp(date) + datetime.timedelta(days=1)
    output.append(out_date.strftime("%Y-%m-%d"))
  else:
    output.append(datetime.datetime.fromtimestamp(date).strftime("%Y-%m-%d"))
tweets_stocks['date'] = output
tweets_stocks = tweets_stocks.sort_values("date", ascending= True)

In [ ]:
#filtrando os tweets para que fiquem somente os relacionados a tesla
tweets_stocks = tweets_stocks.loc[tweets_stocks['ticker_symbol'] == 'TSLA']
tweets_stocks.drop('tweet_id', inplace=True, axis=1)
tweets_stocks.drop('ticker_symbol', inplace=True, axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#trocando o nome das colunas dos tweets do elon musk
elon_tweets['date'] = elon_tweets['date_new']
elon_tweets['like'] = elon_tweets['nlikes']
elon_tweets['comment'] = elon_tweets['nreplies']
elon_tweets['rt'] = elon_tweets['nretweets']
elon_tweets.drop('nlikes', inplace=True, axis=1)
elon_tweets.drop('nreplies', inplace=True, axis=1)
elon_tweets.drop('nretweets', inplace=True, axis=1)
elon_tweets.drop('date_new', inplace=True, axis=1)

In [ ]:
#ajustando um problema de compatibilidade entre as datas das ações e os tweets
def newYear(item):
  if(item == '2020-01-01'):
    return '2020-01-02'
  else:
    return item

tweets_stocks['date'] = tweets_stocks['date'].apply(lambda date: newYear(date))
elon_tweets['date'] = elon_tweets['date'].apply(lambda date: newYear(date))

In [ ]:
#relacionando os tweets do elon musk com as ações da tesla pela data
#ja ajustando os tweets para que caso sejam feitos no periodo que a bolsa esteja fechada, sejam relacionados ao proximo momento que ela abrir
from re import X
from datetime import datetime, timedelta
import numpy as np
stock_days = stocks_tesla['New_Date'].tolist()
elon_days = elon_tweets['date'].tolist()
disorder = True
cicles = 1
output = []

print('tamanho da lista inicial->', len(elon_days))
while disorder == True:
  mistake_list = []
  output = []
  print('resetou ->',len(output))
  for x, item in enumerate(elon_days):
    if item not in stock_days:
      mistake_list.append(item)
      if cicles <= 7:
        example = datetime.strptime(item, '%Y-%m-%d') + timedelta(days=1)
        elon_days[x] =  example.strftime("%Y-%m-%d")
    else:
      output.append(item)
        
  if len(mistake_list) == 0 or cicles >= 7:
    print("Todos os itens tem correspondência")
    print('tamanho da lista final->', len(output))
    disorder = False   
  else :
    print(cicles, '- item sem correspondência ->', len(np.unique(np.array(mistake_list))), 'itens restantes->', len(output))
    cicles = cicles + 1
    disorder = True
tesla_days = output

tamanho da lista inicial-> 10792
resetou -> 0
1 - item sem correspondência -> 425 itens restantes-> 7499
resetou -> 0
2 - item sem correspondência -> 217 itens restantes-> 9057
resetou -> 0
3 - item sem correspondência -> 21 itens restantes-> 10574
resetou -> 0
Todos os itens tem correspondência
tamanho da lista final-> 10792


In [ ]:
elon_tweets['stock_days'] = tesla_days
stocks_tesla = stocks_tesla.set_index("New_Date")
tesla_by_days = stocks_tesla.loc[tesla_days]

#trocando o nome das colunas
elon_tweets_stocks = elon_tweets
elon_tweets_stocks['volume'] = tesla_by_days['volume'].tolist()
elon_tweets_stocks['open'] = tesla_by_days['open_value'].tolist()
elon_tweets_stocks['close'] = tesla_by_days['close_value'].tolist()
elon_tweets_stocks['high'] = tesla_by_days['high_value'].tolist()
elon_tweets_stocks['low'] = tesla_by_days['low_value'].tolist()
elon_tweets_stocks['variation'] = tesla_by_days['variation'].tolist()

elon_tweets_stocks

In [ ]:
#relacionando tweets as ações
stocks = stocks.set_index("day_date")
tweets_days = tweets_stock['date'].tolist()
stocks_by_day = stocks.loc[tweets_days]
#elon_days = elon_tweets['date'].tolist()
#elon_by_day = stocks.loc[elon_days]

In [ ]:
#trocando o nome das colunas
tweets_stocks = tweets_stocks
tweets_stocks['volume'] = stocks_by_day['volume'].tolist()
tweets_stocks['open'] = stocks_by_day['open_value'].tolist()
tweets_stocks['close'] = stocks_by_day['close_value'].tolist()
tweets_stocks['high'] = stocks_by_day['high_value'].tolist()
tweets_stocks['low'] = stocks_by_day['low_value'].tolist()
tweets_stocks['variation'] = stocks_by_day['variation'].tolist()

# tweets_stocks.drop('ticker_symbol', inplace=True, axis=1)
# tweets_stocks.drop('tweet_id', inplace=True, axis=1)

tweets_stocks

,tweet,date,like,comment,rt,volume,open,close,high,low,variation
3239614,Our Penny Stock Picks Gained Over 968% In the ...,2015-01-01,0,0,0,2392947,223.09,222.41,225.6800,222.25,-0.68
3239617,How It's Made: Dream Cars - Tesla Model S. ht...,2015-01-01,0,0,0,2392947,223.09,222.41,225.6800,222.25,-0.68
3239616,"Almost 10,000 Electric Cars Were Sold In China...",2015-01-01,1,0,1,2392947,223.09,222.41,225.6800,222.25,-0.68
3239615,RT @RobertWeinstein: Options Investor Newslett...,2015-01-01,0,0,0,2392947,223.09,222.41,225.6800,222.25,-0.68
3239610,Stock Market Outlook: An Average Return Of 26....,2015-01-01,1,0,0,2392947,223.09,222.41,225.6800,222.25,-0.68
...,...,...,...,...,...,...,...,...,...,...,...
4336187,Tesla must face lawsuit claiming racism at Cal...,2020-01-02,8,0,0,9558386,424.50,430.26,430.6957,421.71,5.76
4336188,"""Crashed the car - car can’t drive, how to get...",2020-01-02,13,3,1,9558386,424.50,430.26,430.6957,421.71,5.76
4336189,Ya $TSLA shorts and longs are much more intell...,2020-01-02,2,2,0,9558386,424.50,430.26,430.6957,421.71,5.76
4336190,$TSLA setting up slowly here for big flag test...,2020-01-02,0,0,0,9558386,424.50,430.26,430.6957,421.71,5.76


In [ ]:
#exportando os dados em csv para uma proxima etapa

compression_opts = dict(method='zip',
                        archive_name='data.csv')  
tweets_stocks.to_csv('tweets_stocks.zip', index=False,
          compression=compression_opts) 

compression_opts = dict(method='zip',
                        archive_name='data.csv')  
elon_tweets_stocks.to_csv('elon_tweets_stocks.zip', index=False,
          compression=compression_opts)  
         